In [ ]:
import os
import numpy as np
import pandas as pd

# Load the dataset from the specified files
def load_music_files(file_indices):
    music_data = []
    for index in file_indices:
        filepath = f'./{index}'  # Construct the file path
        if os.path.exists(filepath):
            with open(filepath, 'r') as file:
                content = file.read()
                music_data.append(content)  # Append the content of the file
    return music_data

file_indices = list(range(931))  # Files 0 to 25
music_files_content = load_music_files(file_indices)

# Display the number of files loaded and the first few contents
print(f"Loaded {len(music_files_content)} music files.")
print("First few contents of the files:")
for i in range(min(3, len(music_files_content))):  # Show first 3 files
    print(f"File {i}: {music_files_content[i][:100]}...")  # Show first 100 characters

Loaded 783 music files.
First few contents of the files:
File 0: 76 _ 81 _ _ _ 83 84 86 84 84 83 83 81 81 80 81 _ _ _ 83 84 86 _ _ _ 88 _ 84 _ _ 83 83 81 81 80 81 _ ...
File 1: 55 _ 55 _ 55 _ 60 _ _ _ 64 _ 55 _ 55 _ 55 _ 60 _ _ _ 64 _ _ _ _ _ _ _ 60 _ 60 _ 59 _ 59 _ 57 _ 57 _ ...
File 2: r _ 55 _ 55 _ 55 _ 60 _ r _ _ _ 64 _ r _ 55 _ 55 _ 55 _ 60 _ r _ _ _ 64 _ r _ _ _ _ _ _ _ r _ 60 _ _...


In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import os

# Load and preprocess the data
def load_and_preprocess_data(file_indices):
    all_sequences = []
    for index in file_indices:
        filepath = f'./{index}'
        if os.path.exists(filepath):
            with open(filepath, 'r') as file:
                sequence = file.read().strip().split()
                sequence = [int(note) if note != '_' and note != 'r' else 0 for note in sequence]
                all_sequences.extend(sequence)
    return all_sequences

# Load the data
file_indices = list(range(26))
music_data = load_and_preprocess_data(file_indices)

# Create sequences for training
def create_sequences(data, seq_length):
    sequences = []
    next_notes = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        next_notes.append(data[i+seq_length])
    return np.array(sequences), np.array(next_notes)

seq_length = 50
X, y = create_sequences(music_data, seq_length)

# Normalize the data
X = X / max(music_data)
y = y / max(music_data)

# Build the model
model = keras.Sequential([
    layers.LSTM(128, input_shape=(seq_length, 1), return_sequences=True),
    layers.LSTM(64),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Reshape X for LSTM input
X = X.reshape((X.shape[0], X.shape[1], 1))

# Train the model
history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

print("Data shape:", X.shape)
print("Number of sequences:", len(X))

# Save the model
model.save('music_generation_model.h5')

# Function to generate music
def generate_music(seed, length):
    generated = list(seed)
    for _ in range(length):
        sequence = np.array(generated[-seq_length:]) / max(music_data)
        sequence = sequence.reshape((1, seq_length, 1))
        next_note = model.predict(sequence)[0][0]
        generated.append(int(next_note * max(music_data)))
    return generated

# Generate a sample
seed = music_data[:seq_length]
generated_music = generate_music(seed, 100)
print("Sample generated music:")
print(generated_music)

Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.1089 - val_loss: 0.1500
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1042 - val_loss: 0.1478
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1057 - val_loss: 0.1541
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1007 - val_loss: 0.1375
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0869 - val_loss: 0.1391
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0761 - val_loss: 0.1141
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0728 - val_loss: 0.1116
Epoch 8/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0719 - val_loss: 0.1091
Epoch 9/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0695 - val_loss: 0.1059
Epoch 10/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0700 - val_loss: 0.1117
Epoch 11/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0709 - val_loss: 0.1052
Epoch 12/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0655 - val_l

Data shape: (3681, 50, 1)
Number of sequences: 3681
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/st

In [ ]:
import pandas as pd

data =  list(generated_music)
df = pd.DataFrame(data, columns=['Values'])

file_name = 'output_data.csv'
df.to_csv(file_name, index=False)

print('File saved as', file_name)

File saved as output_data.csv
